In [1]:
import pandas as pd 

In [2]:
import datetime as dt

In [3]:
import pandas as pd 
df_genres = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\genres.parquet', columns=['genres', 'id', 'release_date'])
df_genres.head()

,genres,id,release_date
0,Action,761140,2018-01-04
1,Casual,761140,2018-01-04
2,Indie,761140,2018-01-04
3,Simulation,761140,2018-01-04
4,Strategy,761140,2018-01-04


In [4]:
df_user = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\users_items.parquet', columns=['user_id', 'playtime_forever', 'item_id'])
df_user.head()

,user_id,playtime_forever,item_id
0,76561197970982479,6,10
1,76561197970982479,0,20
2,76561197970982479,7,30
3,76561197970982479,0,40
4,76561197970982479,0,50


In [5]:
df = df_genres.merge(df_user, how='left', left_on='id', right_on= 'item_id')
df.head()

,genres,id,release_date,user_id,playtime_forever,item_id
0,Action,761140,2018-01-04,NaN,NaN,NaN
1,Casual,761140,2018-01-04,NaN,NaN,NaN
2,Indie,761140,2018-01-04,NaN,NaN,NaN
3,Simulation,761140,2018-01-04,NaN,NaN,NaN
4,Strategy,761140,2018-01-04,NaN,NaN,NaN


In [6]:
df.isna().sum()

genres                167
id                      0
release_date            0
user_id             50122
playtime_forever    50122
item_id             50122
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
df.isna().sum()

genres              0
id                  0
release_date        0
user_id             0
playtime_forever    0
item_id             0
dtype: int64

In [9]:
df.head()

,genres,id,release_date,user_id,playtime_forever,item_id
79,Action,282010,1997-06-30,UTNerd24,5.0,282010.0
80,Action,282010,1997-06-30,I_DID_911_JUST_SAYING,0.0,282010.0
81,Action,282010,1997-06-30,76561197962104795,0.0,282010.0
82,Action,282010,1997-06-30,r3ap3r78,0.0,282010.0
83,Action,282010,1997-06-30,saint556,13.0,282010.0


In [10]:
df['Año'] = df['release_date'].dt.year

In [11]:
df.drop(columns=['id', 'item_id', 'release_date'], inplace=True)
df.head()

,genres,user_id,playtime_forever,Año
79,Action,UTNerd24,5.0,1997
80,Action,I_DID_911_JUST_SAYING,0.0,1997
81,Action,76561197962104795,0.0,1997
82,Action,r3ap3r78,0.0,1997
83,Action,saint556,13.0,1997


In [12]:
df = df.groupby(['genres', 'user_id', 'Año']).agg('sum').reset_index()
df.head()

,genres,user_id,Año,playtime_forever
0,1990's,-Mad-,2012,20.0
1,1990's,117arbiter,2011,205.0
2,1990's,117arbiter,2012,220.0
3,1990's,11XGX,2011,1.0
4,1990's,11XGX,2012,0.0


In [13]:
df['playtime_forever'] = round(df['playtime_forever'] / 60, 2)

In [14]:
df['genres'] = df['genres'].apply(lambda x: x.lower())
df.head()

,genres,user_id,Año,playtime_forever
0,1990's,-Mad-,2012,0.33
1,1990's,117arbiter,2011,3.42
2,1990's,117arbiter,2012,3.67
3,1990's,11XGX,2011,0.02
4,1990's,11XGX,2012,0.00


In [15]:
df.to_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\userforgenre2.parquet')

In [16]:
def UserForGenre(genero: str):
    
    if not isinstance(genero, str):
        return {'Mensaje': 'El género ingresado debe ser una cadena de texto (string)'}
    
    genero = genero.lower()

    try:
        df = pd.read_parquet('c:\\Users\\Sofia\\Desktop\\Clean Data\\userforgenre2.parquet')
        df = df[df['genres'].isin(['action'])].drop(columns='genres')
    except Exception:
        return {'Error': 'Género no encontrado. Ingrese un género válido'}

    usuario_max_horas = df.groupby('user_id').agg({'playtime_forever': 'sum'}).sort_values('playtime_forever').tail(1).index[0]
    df = df[df['user_id'].isin([usuario_max_horas])].reset_index(drop=True)

    resultado = {
        f'Usuario con mas horas jugadas para el género {genero}:': usuario_max_horas,
        'Horas jugadas:': [{'Año:': int(df.loc[i,'Año']), 'Horas:': float(df.loc[i,'playtime_forever'])} for i in range(len(df))]
    }
    del df
    return resultado

In [17]:
UserForGenre('action')

{'Usuario con mas horas jugadas para el género action:': 'Sp3ctre',
 'Horas jugadas:': [{'Año:': 1993, 'Horas:': 0.0},
  {'Año:': 1995, 'Horas:': 3.62},
  {'Año:': 1996, 'Horas:': 0.0},
  {'Año:': 1998, 'Horas:': 0.0},
  {'Año:': 1999, 'Horas:': 0.73},
  {'Año:': 2000, 'Horas:': 1177.4},
  {'Año:': 2001, 'Horas:': 0.22},
  {'Año:': 2002, 'Horas:': 3.97},
  {'Año:': 2003, 'Horas:': 127.88},
  {'Año:': 2004, 'Horas:': 2123.52},
  {'Año:': 2005, 'Horas:': 355.65},
  {'Año:': 2006, 'Horas:': 14.93},
  {'Año:': 2007, 'Horas:': 1879.73},
  {'Año:': 2008, 'Horas:': 3.73},
  {'Año:': 2009, 'Horas:': 1805.43},
  {'Año:': 2010, 'Horas:': 1301.38},
  {'Año:': 2011, 'Horas:': 2581.6},
  {'Año:': 2012, 'Horas:': 6332.55},
  {'Año:': 2013, 'Horas:': 2007.68},
  {'Año:': 2014, 'Horas:': 2178.18},
  {'Año:': 2015, 'Horas:': 5208.52},
  {'Año:': 2016, 'Horas:': 488.3},
  {'Año:': 2017, 'Horas:': 722.12}]}